# 口コミサイトスクレイピング

対象ページ  
[日本酒物語 日本酒ランキング（人数）](http://www.sakeno.com/followrank/) 

### 1. 使用するライブラリの読み込み


In [1]:
# ファイル操作
import glob
import csv

# データ処理
import pandas as pd
import numpy as np

# クローラー
import time
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import urllib.parse as urlparse

### 2. 定数設定

　ここでは取得対象URLやスクレイピング待ち時間設定、出力先設定などを定義

In [2]:
# 日本酒物語 日本酒ランキング（人数）の URL
FOLLOWRANK_URL = 'http://www.sakeno.com/followrank/'

# クロール時の待ち時間
WAIT_TIME = 3

# 銘柄マスタの出力先
MEIGARA_MASTER_PATH = '../data/meigara_maseter.csv'
# 銘柄評価スコアの出力先ディレクトリ
MEIGARA_SCORES_DIR = '../data/meigara_scores/'
# 銘柄コメントの出力先ディレクトリ
MEIGARA_COMMENTS_DIR = '../data/meigara_comments/'


### 3. ランキング一覧の要素の取得と銘柄マスタ作成

　ランキング一覧ページから各銘柄のデータを取得し、マスタデータを作成

In [3]:
# パースと整形
def parse_tr(tr):
    # 順位
    tds = tr.find_all('td')
    rank = int(tds[0].get_text().split('位')[0]) 

    # 銘柄
    a = tds[1].find('a')
    meigara = a.get_text()
    detail_url = a.get('href')
    yomi = tds[1].find('div').string

    # 酒造
    location = tds[2].find_all('a')
    kuramoto = location[0].string
    prefecture = location[1].string
    city = location[2].string
    
    tr_l = [
        rank, meigara, yomi,
        kuramoto, prefecture, 
        city,detail_url
    ]
    
    return tr_l

In [4]:
# 1ページ分の情報取得とData frame作成
def get_ranking_info(url):
    # 連続アクセス時の負荷軽減
    time.sleep(WAIT_TIME)
    
    response = requests.get(url)

    # 正しく取得できたかどうか HTTP ステータスコードで確認
    if not response.status_code == 200:
        raise ValueError('Invalid response')
    else:
        print(url,'OK.')
    soup = BeautifulSoup(response.text, 'lxml')
    table = soup.body.find('table')

    # 先頭のゴミをカット
    trs = table.find_all('tr')[2:]
    
    ranking_list = [parse_tr(tr) for tr in trs]
    
    meigara_master_df = pd.DataFrame(
        ranking_list,
        columns=['rank', 'meigara', 'yomi',
                 'kuramoto', 'prefecture', 'city',
                 'detail_url']
    )
    
    return meigara_master_df

In [5]:
# 全ページ（4ページ分）の情報取得
for i in range(1,5):
    _rank = get_ranking_info(FOLLOWRANK_URL+'?p='+str(i))
    if i==1:
        meigara_master_df = _rank
    else:
        meigara_master_df = pd.concat([meigara_master_df, _rank])

http://www.sakeno.com/followrank/?p=1 OK.
http://www.sakeno.com/followrank/?p=2 OK.
http://www.sakeno.com/followrank/?p=3 OK.
http://www.sakeno.com/followrank/?p=4 OK.


In [6]:
# インデックスリセット
meigara_master_df.reset_index(inplace=True,drop=True)

# ユニークな連番 ID を追加
meigara_master_df["meigara_id"] = meigara_master_df.index.to_series() + 1
meigara_master_df = meigara_master_df[
    ["meigara_id",
     "rank", "meigara", "yomi",
     "kuramoto", "prefecture", "city",
     "detail_url"]
]

In [7]:
# 銘柄マスタデータのCSV出力
meigara_master_df.to_csv(
    MEIGARA_MASTER_PATH,
    encoding="utf-8",
    sep=",",
    index=False
)

In [8]:
# 銘柄マスタデータ
meigara_master_df.head(3)

,meigara_id,rank,meigara,yomi,kuramoto,prefecture,city,detail_url
0,1,1,獺祭,だっさい,旭酒造（山口県）,山口県,岩国市,https://www.sakeno.com/meigara/931/
1,2,2,醸し人九平次,かもしびとくへいじ,萬乗醸造,愛知県,名古屋市緑区,https://www.sakeno.com/meigara/735/
2,3,3,田酒,でんしゅ,西田酒造店,青森県,青森市,https://www.sakeno.com/meigara/11/


### 4. 銘柄マスタを元に詳細情報の取得と整形

　銘柄マスタのURLを元に評価ページと口コミページの情報を取得し整形する

#### 4.1. 評価ページの取得と整形用の処理

In [9]:
# 数値による評価データ取得関数
def parse_scores_table(soup, meigara_id):
    scores_table = soup.body.find_all("form")[2]
    trs = scores_table.find_all("tr")

    # 味
    aji = trs[2].find_all("span")
    aji_good = int(aji[0].string)
    aji_bad = int(aji[1].string)

    # 香り
    kaori = trs[3].find_all("span")
    kaori_good = int(kaori[0].string)
    kaori_bad = int(kaori[1].string)

    # 濃さ
    kosa = trs[4].find_all("span")
    kosa_good = int(kosa[0].string)
    kosa_bad = int(kosa[1].string)

    # 価格
    kakaku = trs[5].find_all("span")
    kakaku_good = int(kakaku[0].string)
    kakaku_bad = int(kakaku[1].string)

    # デザイン
    design = trs[6].find_all("span")
    design_good = int(design[0].string)
    design_bad = int(design[1].string)

    score_li = [
        [meigara_id, "味", aji_good, aji_bad],
        [meigara_id, "香り", kaori_good, kaori_bad],
        [meigara_id, "濃さ", kosa_good, kosa_bad],
        [meigara_id, "価格", kakaku_good, kakaku_bad],
        [meigara_id, "デザイン", design_good, design_bad]
    ]

    score_df = pd.DataFrame(
        score_li,
        columns=["meigara_id", "name", "good_score", "bad_score"]
    )
    
    return score_df

In [10]:
# コメント一覧取得関数
def parse_comments_table(soup, meigara_id):
    reviews_table = soup.body.find_all("table")[-1]

    dts = [
        [meigara_id,
         int(dt.contents[0].get("name").replace("voice", "")),
         dt.contents[3].string]
        for dt
        in reviews_table.find_all("dt")
    ]
    dts_df = pd.DataFrame(
        dts,
        columns=["meigara_id", "toukou_id", "title"]
    )
    
    # <dd>〜</dd> の処理
    dds = [
        [dd.contents[-1].text.split("（")[1].split("）")[0],
         dd.contents[-1].find("a").string,
         dd.contents[-2].replace("\n", " ")]
        for dd
        in reviews_table.find_all("dd")
    ]
    dds_df = pd.DataFrame(
        dds,
        columns=["created_at", "user_name", "text"]
    )
    dds_df["created_at"] = dds_df["created_at"].apply(
        lambda x: datetime.strptime(x, '%Y年%m月%d日 %H時%M分%S秒')
        )
    
    # 結合
    comments_df = pd.concat([dts_df, dds_df], axis=1)

    return comments_df

In [11]:
# すべての詳細ページからデータを取得するための関数
def parse_maigara_detail_page(row):
    print(
        datetime.now().isoformat(sep=" "),
        row["meigara_id"],
        row["meigara"]
    )
    
    # 連続アクセス時の負荷軽減
    time.sleep(WAIT_TIME)
    
    # クローリング
    response = requests.get(row["detail_url"])
    if not response.status_code == 200:
        raise ValueError("Invalid response")
    #response.encoding = 'euc_jp'
    # ゴミとなる文字群を除去
    preprocessed_html_string = response.text.replace("<br>", "\n")
    preprocessed_html_string = preprocessed_html_string.replace("\r", "")
    preprocessed_html_string = preprocessed_html_string.replace("　", " ")
    soup = BeautifulSoup(preprocessed_html_string, "lxml")
    
    # 評価スコアの取得 & 出力
    score_df = parse_scores_table(soup, row["meigara_id"])
    scores_path = MEIGARA_SCORES_DIR + str(row["meigara_id"]) + ".csv"
    score_df.to_csv(
        scores_path,
        encoding="utf-8",
        sep=",",
        index=False,
        quoting=csv.QUOTE_NONNUMERIC
    )
    
    return

#### 4.2. コメントページの取得と整形用の処理

In [12]:
def get_max_page(row):
    # 連続アクセス時の負荷軽減
    time.sleep(WAIT_TIME)
    
    # クローリング
    url = urlparse.urljoin(row["detail_url"], 'voice')
    response = requests.get(url)
    if not response.status_code == 200:
        raise ValueError("Invalid response")
    soup = BeautifulSoup(response.text, "lxml")
    
    next_page = soup.body.select("li.next a")
    
    if len(next_page) == 2:
        max_page = int(next_page[1].get("href").replace("?p=", ""))
    elif len(next_page) == 1:
        max_page = int(next_page[0].get("href").replace("?p=", ""))
    else:
        max_page = 1
    
    return max_page


In [13]:
#  コメントページの取得
def get_comment_page(url):
    # 連続アクセス時の負荷軽減
    time.sleep(WAIT_TIME)
    
    # クローリング
    response = requests.get(url)
    if not response.status_code == 200:
        raise ValueError("Invalid response")
    
    # ゴミとなる文字群を除去
    preprocessed_html_string = response.text.replace("<br>", "\n")
    preprocessed_html_string = preprocessed_html_string.replace("\r", "")
    preprocessed_html_string = preprocessed_html_string.replace("　", " ")
    soup = BeautifulSoup(preprocessed_html_string, "lxml")
    
    return soup

In [14]:
# コメント一覧取得関数
def parse_comments_table(soup):
    touroku_id = [n.contents[0].replace("日本酒口コミNo.", "") for n in soup.body.find_all("span",class_="voiceno")]
    title = [n.contents[1].get_text().replace("\xa0>\xa0", ":") for n in soup.body.find_all("h4",class_="voicetitle")]
    voicenaiyo = [n.get_text() for n in soup.body.find_all("div",class_="voicenaiyo")]
    usrname = [n.contents[0].get_text() for n in soup.body.find_all("div",class_="voicemember")]
    update = [n.contents[2].get_text() for n in soup.body.find_all("div",class_="voicemember")]

    commdf = pd.DataFrame(list(zip(touroku_id ,title,voicenaiyo,usrname,update)), columns=['touroku_id ','title', 'voicenaiyo', 'user_name', 'created_at'])
    commdf["created_at"] = commdf["created_at"].apply(lambda x: datetime.strptime(x, '（%Y年%m月%d日 %H時%M分%S秒）'))
    
    return commdf

In [15]:
# すべての詳細ページからデータを取得するための関数
def parse_maigara_detail_comments(row):
    
    max_page = get_max_page(row)
    page_num = 1
    
    while page_num <= max_page:
        print(datetime.now().isoformat(sep=" "),
              row["meigara_id"],
              row["meigara"],
              'コメント取得中',
              str(page_num)+'/'+str(max_page)
             )
        
        url = urlparse.urljoin(row["detail_url"], 'voice', '?p='+str(page_num))
        soup = get_comment_page(url)
        _df = parse_comments_table(soup)
        
        if page_num == 1:
            comments_df = _df.copy()
        else:
            comments_df = pd.concat([comments_df, _df])
        page_num += 1
 
    # 評価コメントの取得 & 出力
    comments_df['meigara_id'] = row['meigara_id']
    comments_df= comments_df[['meigara_id','touroku_id ','title', 'voicenaiyo', 'user_name', 'created_at']]
    comments_path = MEIGARA_COMMENTS_DIR + str(row["meigara_id"]) + ".csv"
    comments_df.to_csv(
        comments_path,
        encoding="utf-8",
        sep=",",
        index=False,
        quoting=csv.QUOTE_NONNUMERIC
    )
    return

#### 4.3. 取得整形処理実行

In [ ]:
for idx, row in meigara_master_df.iterrows():
    parse_maigara_detail_page(row)
    parse_maigara_detail_comments(row)

2021-02-18 22:50:02.885765 1 獺祭
2021-02-18 22:50:11.075726 1 獺祭 コメント取得中 1/14
2021-02-18 22:50:14.940341 1 獺祭 コメント取得中 2/14
2021-02-18 22:50:18.772403 1 獺祭 コメント取得中 3/14
2021-02-18 22:50:22.735544 1 獺祭 コメント取得中 4/14
2021-02-18 22:50:26.587229 1 獺祭 コメント取得中 5/14
2021-02-18 22:50:30.460327 1 獺祭 コメント取得中 6/14
2021-02-18 22:50:34.386971 1 獺祭 コメント取得中 7/14
2021-02-18 22:50:38.306937 1 獺祭 コメント取得中 8/14
2021-02-18 22:50:42.103497 1 獺祭 コメント取得中 9/14
2021-02-18 22:50:45.955404 1 獺祭 コメント取得中 10/14
2021-02-18 22:50:49.867736 1 獺祭 コメント取得中 11/14
2021-02-18 22:50:53.715069 1 獺祭 コメント取得中 12/14
2021-02-18 22:50:57.587474 1 獺祭 コメント取得中 13/14
2021-02-18 22:51:01.582058 1 獺祭 コメント取得中 14/14
2021-02-18 22:51:05.390216 2 醸し人九平次
2021-02-18 22:51:13.339246 2 醸し人九平次 コメント取得中 1/12
2021-02-18 22:51:17.213729 2 醸し人九平次 コメント取得中 2/12
2021-02-18 22:51:21.060548 2 醸し人九平次 コメント取得中 3/12
2021-02-18 22:51:24.900865 2 醸し人九平次 コメント取得中 4/12
2021-02-18 22:51:28.939805 2 醸し人九平次 コメント取得中 5/12
2021-02-18 22:51:32.797791 2 醸し人九平次 コメント取得中 6/12
202